In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [3]:
def load_data(dataset_path):
    data = []
    labels = []
    authors = []  # List to store author names
    for author in os.listdir(dataset_path):
        author_folder = os.path.join(dataset_path, author)
        if os.path.isdir(author_folder):
            authors.append(author)  # Add author to the list
            for poem in os.listdir(author_folder):
                poem_path = os.path.join(author_folder, poem)
                if poem_path.endswith('.txt'):
                    with open(poem_path, 'r', encoding='utf-8') as file:
                        text = file.read()
                        data.append(text)
                        labels.append(author)
    return data, labels, authors

dataset_path = 'dataset'
data, labels, authors = load_data(dataset_path)


# retrive/print the names of the authors
print("Authors:", authors)


Authors: ['AbuAlAtahiya', 'AbuAlFadlAldarimi', 'AbuFirasAlHamdani', 'AbuNuwas', 'AliAlTuhami', 'AliIbnAlJahm', 'AlMutanabbi', 'IbnAlFarid', 'UlayyaBintAlMahdi']
